# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [77]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [78]:
# read in the .csv
ct_data = "..\output_data\cities.csv"
city_df = pd.read_csv(ct_data)
del city_df['Unnamed: 0']
city_df[['Latitude' , 'Longitude']] = city_df[['Latitude', 'Longitude']].apply(pd.to_numeric)
city_df.head()

,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Country,Date of Weather
0,Donegal,54.6500,-8.1167,68.04,63,90,3.56,IE,09-12-2021
1,Ushuaia,-54.8000,-68.3000,42.46,70,75,0.00,AR,09-12-2021
2,Bairiki Village,1.3292,172.9752,82.06,74,97,21.88,KI,09-12-2021
3,George Town,5.4112,100.3354,80.89,82,20,0.00,MY,09-12-2021
4,Adrar,20.5022,-10.0711,104.59,15,89,10.74,MR,09-12-2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [79]:
import gmaps

# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = city_df["Humidity"].astype(float)

# Create the heatmaps (used example form anthony)
fig = gmaps.figure(center=[0,0], zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [80]:
# creating a new dataframe for only those cities that are between 70 and 80 degrees, windspeed less than 10
ideal_city = city_df.loc[(city_df['Tempature']>=70) & (city_df['Tempature'] <=80)]
ideal_city = ideal_city.loc[(ideal_city['Windspeed'] <= 10)]
ideal_city.dropna()
ideal_city.head()

,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Country,Date of Weather
6,Isangel,-19.5500,169.2667,72.25,83,51,8.41,VU,09-12-2021
7,Bethel,41.3712,-73.4140,77.20,74,1,3.00,US,09-12-2021
10,Tanete,-3.9443,119.7848,75.07,93,100,0.07,ID,09-12-2021
13,Ribeira Grande,38.5167,-28.7000,73.80,60,40,8.05,PT,09-12-2021
15,Rzhanitsa,53.4284,33.9230,72.05,60,81,3.27,RU,09-12-2021


In [81]:
# stor into new hotel_df
hotel_df = ideal_city.reset_index(drop=True)
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Country,Date of Weather,Hotel Name
0,Isangel,-19.5500,169.2667,72.25,83,51,8.41,VU,09-12-2021,
1,Bethel,41.3712,-73.4140,77.20,74,1,3.00,US,09-12-2021,
2,Tanete,-3.9443,119.7848,75.07,93,100,0.07,ID,09-12-2021,
3,Ribeira Grande,38.5167,-28.7000,73.80,60,40,8.05,PT,09-12-2021,
4,Rzhanitsa,53.4284,33.9230,72.05,60,81,3.27,RU,09-12-2021,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [82]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a parameters dictionary
params = {
    "type" : "hotel",
    "keyword": "hotel",
    "radius": "5000",    
    "key": g_key
}



In [83]:
# Create a loop through each city and lat/lon in the datafraem
for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row["Longitude"]
    city_name = row["City"]

    params["location"] = f"{lat},{lng}"

     # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 0: Isangel.
Closest hotel in Isangel is Tanna Ikamir Guest House.
------------
Retrieving Results for Index 1: Bethel.
Closest hotel in Bethel is La Quinta Inn & Suites by Wyndham Danbury.
------------
Retrieving Results for Index 2: Tanete.
Closest hotel in Tanete is Hotel Tri Multi.
------------
Retrieving Results for Index 3: Ribeira Grande.
Closest hotel in Ribeira Grande is Azul Singular.
------------
Retrieving Results for Index 4: Rzhanitsa.
Missing field/result... skipping.
------------
Retrieving Results for Index 5: Mount Isa.
Closest hotel in Mount Isa is Redearth Boutique Hotel.
------------
Retrieving Results for Index 6: Kapaa.
Closest hotel in Kapaa is The ISO.
------------
Retrieving Results for Index 7: Somerville.
Closest hotel in Somerville is The Revolution Hotel.
------------
Retrieving Results for Index 8: Sri Aman.
Closest hotel in Sri Aman is Seri Simanggang Hotel.
------------
Retrieving Results for Index 9: Rikitea.
Closest hotel i

In [84]:
hotel_df.head()


,City,Latitude,Longitude,Tempature,Humidity,Cloudiness,Windspeed,Country,Date of Weather,Hotel Name
0,Isangel,-19.5500,169.2667,72.25,83,51,8.41,VU,09-12-2021,Tanna Ikamir Guest House
1,Bethel,41.3712,-73.4140,77.20,74,1,3.00,US,09-12-2021,La Quinta Inn & Suites by Wyndham Danbury
2,Tanete,-3.9443,119.7848,75.07,93,100,0.07,ID,09-12-2021,Hotel Tri Multi
3,Ribeira Grande,38.5167,-28.7000,73.80,60,40,8.05,PT,09-12-2021,Azul Singular
4,Rzhanitsa,53.4284,33.9230,72.05,60,81,3.27,RU,09-12-2021,


In [85]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [86]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))